## Importing Modules

In [2]:
import os
import gc
gc.enable()
import glob
import random
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import lr_scheduler
from tensorflow.keras.utils import pad_sequences
from sklearn.metrics import accuracy_score
import lzma
import pickle
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 150)

def set_seed(seed_value=23, use_cuda=True):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Paths Settings

In [3]:
class Configuration:
    CREATE_DATA = True
    sequence_length = 5000
    saved_path = ''

train_files_path = './Train_data'
test_files_path = './Test_data'
result_path = './Result.csv'

## Loading of Dataframes

In [4]:
%%time

def read_file_data(f):
    try:
        df = pd.read_csv(f)
        df['ID']=f.split('/')[-1].split('.')[0]
        return df
    except: pass

train_files = glob.glob(train_files_path+'/**')
dfs = []
for f in tqdm(train_files):
    dfs.append(read_file_data(f))
train = pd.concat(dfs).fillna(0)
del dfs
gc.collect()


test_files = glob.glob(test_files_path+'/**')
dfs = []
for f in tqdm(test_files):
    dfs.append(read_file_data(f))
test = pd.concat(dfs).fillna(0)
del dfs
gc.collect()

ss = pd.read_csv(result_path)

display(train, test, ss)

100%|██████████| 19/19 [00:12<00:00,  1.50it/s]


,time,portPktIn,portPktOut,qSize,label,ID
0,0,0.0,0.0,0,10,Train11
1,1,0.0,0.0,0,10,Train11
2,2,0.0,0.0,0,10,Train11
3,3,0.0,0.0,0,10,Train11
4,4,235172.0,235172.0,0,10,Train11
...,...,...,...,...,...,...
141995,141995,0.0,0.0,0,2,Train75
141996,141996,0.0,0.0,0,2,Train75
141997,141997,0.0,0.0,0,2,Train75
141998,141998,0.0,0.0,0,2,Train75


,time,portPktIn,portPktOut,qSize,ID
0,0,317527.0,317527.0,0,Test1
1,1,886571.0,882427.0,4144,Test1
2,2,210899.0,214499.0,544,Test1
3,3,480175.0,480719.0,0,Test1
4,4,157670.0,157670.0,0,Test1
...,...,...,...,...,...
116995,116995,0.0,0.0,0,Test7
116996,116996,0.0,0.0,0,Test7
116997,116997,0.0,0.0,0,Test7
116998,116998,0.0,0.0,0,Test7


,ID,Target
0,test0_0,0
1,test0_1,0
2,test0_2,0
3,test0_3,0
4,test0_4,0
...,...,...
2337995,test18_144995,0
2337996,test18_144996,0
2337997,test18_144997,0
2337998,test18_144998,0


CPU times: user 8.98 s, sys: 1.46 s, total: 10.4 s
Wall time: 1min 38s


In [5]:
display(train['ID'].value_counts())

Train59    149000
Train74    148000
Train18    148000
Train61    148000
Train13    147000
            ...  
Train23    103000
Train20    102000
Train71    102000
Train65    100000
Train22    100000
Name: ID, Length: 78, dtype: int64

In [6]:
display(test['ID'].value_counts())

Test18    145000
Test12    144000
Test6     141000
Test14    139000
Test16    137000
Test2     136000
Test3     131000
Test0     129000
Test15    128000
Test10    122000
Test9     119000
Test7     117000
Test1     113000
Test4     108000
Test11    108000
Test5     107000
Test13    106000
Test8     105000
Test17    103000
Name: ID, dtype: int64

In [7]:
display(train['label'].nunique())

12

## Create Train-Test Arrays

In [8]:
train_ids = list(train['ID'].unique())
train_ids, val_ids = train_test_split(train_ids, random_state=1, test_size=0.15)
print(len(train_ids), len(val_ids))

sequence_length = Configuration.sequence_length
train = train.sort_values('time').reset_index(drop=True)

train['sequence_id'] = train['time'] // sequence_length
train['sequence_id'] = train['sequence_id'].astype(str)
train['sequence_id_ID'] = train['sequence_id'] + '_' + train['ID']
display(train)

labels = np.array(list(set(train['label'].unique())))
print(labels)

66 12


,time,portPktIn,portPktOut,qSize,label,ID,sequence_id,sequence_id_ID
0,0,0.0,0.0,0,10,Train11,0,0_Train11
1,0,0.0,0.0,0,1,Train7,0,0_Train7
2,0,0.0,0.0,0,3,Train23,0,0_Train23
3,0,525952.0,525952.0,0,6,Train31,0,0_Train31
4,0,476175.0,476175.0,0,7,Train26,0,0_Train26
...,...,...,...,...,...,...,...,...
9657995,148995,0.0,0.0,0,10,Train59,29,29_Train59
9657996,148996,0.0,0.0,0,10,Train59,29,29_Train59
9657997,148997,0.0,0.0,0,10,Train59,29,29_Train59
9657998,148998,0.0,0.0,0,10,Train59,29,29_Train59


[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [9]:
%%time

train_df = train.copy()
xtrain_df = train_df[train_df['ID'].isin(train_ids)].reset_index(drop=True)
xval_df = train_df[train_df['ID'].isin(val_ids)].reset_index(drop=True)

features = ['time','portPktIn','portPktOut','qSize']
scaler = RobustScaler()
xtrain_df[features] = scaler.fit_transform(xtrain_df[features])
xval_df[features] = scaler.transform(xval_df[features])

xtrain_df.fillna(0,inplace=True)
xval_df.fillna(0,inplace=True)

xtrain_df.replace([np.inf,-np.inf],0,inplace=True)
xval_df.replace([np.inf,-np.inf],0,inplace=True)

del train,train_df
gc.collect()

if Configuration.CREATE_DATA:
    xtrain_idids = xtrain_df['sequence_id_ID'].unique()
    xtrain_data = []
    xtrain_targets = []
    for d in tqdm(xtrain_idids):
        data = xtrain_df[xtrain_df['sequence_id_ID']==d].sort_values('time')
        inputs = data[features].values
        inputs = pad_sequences([inputs], maxlen=sequence_length, dtype='float',
                               padding='post', value=-1)[0,:,:]
        xtrain_data.append(inputs)

        targets = data['label'].values
        targets = pad_sequences([targets], maxlen=sequence_length, dtype='int',
                                padding='post', value=max(labels)+1)[0]
        xtrain_targets.append(targets)

    xval_idids = xval_df['sequence_id_ID'].unique()
    xval_data = []
    xval_targets = []
    for d in tqdm(xval_idids):
        data = xval_df[xval_df['sequence_id_ID']==d].sort_values('time')
        inputs = data[features].values
        inputs = pad_sequences([inputs], maxlen=sequence_length, dtype='float',
                               padding='post', value=-1)[0,:,:]
        xval_data.append(inputs)

        targets = data['label'].values
        targets = pad_sequences([targets], maxlen=sequence_length, dtype='int',
                                padding='post', value=max(labels)+1)[0]
        xval_targets.append(targets)


    xtrain_data = np.array(xtrain_data)
    xval_data = np.array(xval_data)

    xtrain_targets = np.array(xtrain_targets)
    xval_targets = np.array(xval_targets)


    with open('xtrain_data.npy','wb') as f:
        np.save(f, xtrain_data)

    with open('xval_data.npy','wb') as f:
        np.save(f, xval_data)

    with open('xtrain_targets.npy','wb') as f:
        np.save(f, xtrain_targets)

    with open('xval_targets.npy','wb') as f:
        np.save(f, xval_targets)

else:

    saved_path = Configuration.saved_path

    with open(saved_path+'xtrain_data.npy','rb') as f:
        xtrain_data = np.load(f)

    with open(saved_path+'xval_data.npy','rb') as f:
        xval_data = np.load(f)

    with open(saved_path+'xtrain_targets.npy','rb') as f:
        xtrain_targets = np.load(f)

    with open(saved_path+'xval_targets.npy','rb') as f:
        xval_targets = np.load(f)

100%|██████████| 316/316 [00:55<00:00,  5.66it/s]


CPU times: user 17min 56s, sys: 7.19 s, total: 18min 3s
Wall time: 18min 12s


## Modelling

In [10]:
class Sequence_Dataset(Dataset):
    """ Training Dataloader"""
    def __init__(self, data, targets):

        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.data[idx]
        targets = self.targets[idx]

        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.long)

class Sequence_Dataset_Inference(Dataset):
    """ Inference Dataloader"""
    def __init__(self, data):

        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.data[idx]

        return torch.tensor(inputs, dtype=torch.float32)

def save_checkpoint(checkpoint, filename):
    torch.save(checkpoint, filename)
    print(f"\n--> Saved checkpoint: {filename.split('.')[0]}")

def load_checkpoint(filename, model):
    model.load_state_dict(torch.load(filename)['state_dict'])
    return model

class RNN_Model(nn.Module):
    def __init__(self, input_size, num_classes, hidden_size, num_layers):
        super().__init__()

        self.Lstm_layer_1 = nn.GRU(input_size=input_size,
                                    hidden_size=hidden_size,
                                    num_layers=num_layers,
                                    bidirectional=True,
                                    batch_first=True)

        self.Output = nn.Linear(in_features=self.Lstm_layer_1.hidden_size*2, out_features=num_classes)

    def forward(self, inputs):
        lstm_1_seq, _ = self.Lstm_layer_1(inputs)
        output = self.Output(lstm_1_seq)
        return output

In [11]:
batch_size = 16

train_ds = Sequence_Dataset(xtrain_data, xtrain_targets)
val_ds = Sequence_Dataset(xval_data, xval_targets)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

torch.cuda.empty_cache()
gc.collect()

0

In [12]:
%%time
num_classes = len(labels)+1
input_size = len(features)
hidden_size = 16
num_layers = 1

model = RNN_Model(input_size, num_classes, hidden_size, num_layers).to(device)

criterion = nn.CrossEntropyLoss().to(device)

for x,y in train_loader:
    break

output = model(x.to(device))
loss = criterion(output.view(-1, num_classes), y.to(device).view(-1))
print(loss)
_, predicted = torch.max(output, 2)
print(output.shape)
print(predicted)
print(predicted.shape)

tensor(2.6310, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([16, 5000, 13])
tensor([[3, 3, 3,  ..., 9, 9, 3],
        [3, 3, 3,  ..., 9, 9, 9],
        [3, 3, 3,  ..., 9, 9, 9],
        ...,
        [2, 2, 2,  ..., 2, 2, 2],
        [3, 2, 2,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 9, 9, 9]], device='cuda:0')
torch.Size([16, 5000])
CPU times: user 2.38 s, sys: 1.28 s, total: 3.67 s
Wall time: 12.1 s


## Training of Model

In [13]:
%%time

epochs = 200
batch_size = 4
hidden_size = 64
num_layers = 3

train_ds = Sequence_Dataset(xtrain_data, xtrain_targets)
val_ds = Sequence_Dataset(xval_data, xval_targets)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

torch.cuda.empty_cache()
gc.collect()

input_size = len(features)
num_classes = len(labels)+1
model = RNN_Model(input_size, num_classes, hidden_size, num_layers).to(device)

model_filename = 'network_traffic_model.pth'
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=4, verbose=True, min_lr=1e-6)

best_val_accuracy = 0
best_val_targets = []
best_val_preds = []
epochs_without_improvement = 0
patience = 10

for epoch in range(epochs):
    dataset_size = 0
    running_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train ')
    for step,(inputs, targets) in pbar:
        inputs,targets = inputs.to(device), targets.to(device)

        batch_size = inputs.size(0)

        outputs = model(inputs)
        loss = criterion(outputs.view(-1, num_classes), targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                         lr=f'{current_lr:0.5f}',
                         gpu_mem=f'{mem:0.2f} GB')

    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    with torch.no_grad():
        all_predictions = []
        all_targets = []

        dataset_size = 0
        running_loss = 0.0
        pbar = tqdm(enumerate(val_loader), total=len(val_loader), desc='Valid ')
        for step, (inputs, targets) in pbar:
            inputs,targets = inputs.to(device), targets.to(device)
            batch_size = inputs.size(0)

            outputs = model(inputs)

            loss = criterion(outputs.view(-1, num_classes), targets.view(-1))
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
            current_lr = optimizer.param_groups[0]['lr']
            pbar.set_postfix(val_loss=f'{epoch_loss:0.4f}',
                             lr=f'{current_lr:0.5f}',
                             gpu_mem=f'{mem:0.2f} GB')

            _, predicted = torch.max(outputs, 2)
            all_predictions.extend(predicted.cpu().numpy().ravel())
            all_targets.extend(targets.cpu().numpy().ravel())

    all_targets_ = np.array(all_targets)
    all_predictions_ = np.array(all_predictions)

    all_predictions_ = all_predictions_[np.where(all_targets_!=12)]
    all_targets_ = all_targets_[np.where(all_targets_!=12)]

    val_accuracy = accuracy_score(all_targets_, all_predictions_)
    print(f"Epoch {epoch+1}/{epochs}, Accuracy Score: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_val_targets = all_targets_
        best_val_preds = all_predictions_
        epochs_without_improvement = 0
        checkpoint = {'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint=checkpoint, filename=model_filename)
    else:
        epochs_without_improvement +=1

    if epochs_without_improvement == patience:
        break

    scheduler.step(val_accuracy)

    torch.cuda.empty_cache()
    gc.collect()

del xtrain_data, xtrain_targets, xval_data, xval_targets
torch.cuda.empty_cache()
gc.collect()

Valid : 100%|██████████| 79/79 [00:02<00:00, 29.80it/s, gpu_mem=0.39 GB, lr=0.00100, val_loss=1.1476]


Epoch 1/200, Accuracy Score: 0.5622

--> Saved checkpoint: network_traffic_model


Valid : 100%|██████████| 79/79 [00:02<00:00, 32.45it/s, gpu_mem=0.39 GB, lr=0.00100, val_loss=0.9200]


Epoch 2/200, Accuracy Score: 0.6346

--> Saved checkpoint: network_traffic_model


Valid : 100%|██████████| 79/79 [00:02<00:00, 35.08it/s, gpu_mem=0.39 GB, lr=0.00100, val_loss=0.8567]


...

Valid : 100%|██████████| 79/79 [00:02<00:00, 35.19it/s, gpu_mem=0.39 GB, lr=0.00049, val_loss=0.6849]


Epoch 33/200, Accuracy Score: 0.7228


Valid : 100%|██████████| 79/79 [00:02<00:00, 30.65it/s, gpu_mem=0.39 GB, lr=0.00049, val_loss=0.6647]


Epoch 34/200, Accuracy Score: 0.7298
CPU times: user 16min 36s, sys: 7.82 s, total: 16min 44s
Wall time: 17min 4s


1785

In [14]:
print('BEST VAL ACCURACY: ', np.round(best_val_accuracy,5))

BEST VAL ACCURACY:  0.74172


## Inference

In [15]:
test_files = glob.glob(test_files_path+'/**')
dfs = []
for f in tqdm(test_files):
    dfs.append(read_file_data(f))
test = pd.concat(dfs).fillna(0)
del dfs
gc.collect()

test['time'] = test['time'].astype(int).astype(str)
test['ID2'] = test['ID'].str.replace('T','t') +"_"+ test['time']
test['time'] = test['time'].astype(int)

display(test)

100%|██████████| 19/19 [00:01<00:00, 18.18it/s]


,time,portPktIn,portPktOut,qSize,ID,ID2
0,0,317527.0,317527.0,0,Test1,test1_0
1,1,886571.0,882427.0,4144,Test1,test1_1
2,2,210899.0,214499.0,544,Test1,test1_2
3,3,480175.0,480719.0,0,Test1,test1_3
4,4,157670.0,157670.0,0,Test1,test1_4
...,...,...,...,...,...,...
116995,116995,0.0,0.0,0,Test7,test7_116995
116996,116996,0.0,0.0,0,Test7,test7_116996
116997,116997,0.0,0.0,0,Test7,test7_116997
116998,116998,0.0,0.0,0,Test7,test7_116998


In [16]:
test_df = test.copy()
test_df['sequence_id'] = test_df['time'] // sequence_length
test_df['sequence_id'] = test_df['sequence_id'].astype(str)
test_df['sequence_id_ID'] = test_df['sequence_id'] + '_' + test_df['ID']
test_df[features] = scaler.transform(test_df[features])
display(test_df)

,time,portPktIn,portPktOut,qSize,ID,ID2,sequence_id,sequence_id_ID
0,-0.999996,-1.613716,-1.674106,-0.214286,Test1,test1_0,0,0_Test1
1,-0.999980,-0.597341,-0.659047,-0.142857,Test1,test1_1,0,0_Test1
2,-0.999963,-1.804166,-1.859236,-0.204909,Test1,test1_2,0,0_Test1
3,-0.999947,-1.323209,-1.380869,-0.214286,Test1,test1_3,0,0_Test1
4,-0.999931,-1.899238,-1.961351,-0.214286,Test1,test1_4,0,0_Test1
...,...,...,...,...,...,...,...,...
116995,0.907534,-2.180854,-2.244666,-0.214286,Test7,test7_116995,23,23_Test7
116996,0.907550,-2.180854,-2.244666,-0.214286,Test7,test7_116996,23,23_Test7
116997,0.907566,-2.180854,-2.244666,-0.214286,Test7,test7_116997,23,23_Test7
116998,0.907583,-2.180854,-2.244666,-0.214286,Test7,test7_116998,23,23_Test7


In [17]:
xtest_idids = test_df['sequence_id_ID'].unique()

xtest_data = []
test_id_data = []
for d in tqdm(xtest_idids):
    data = test_df[test_df['sequence_id_ID']==d].sort_values('time')
    inputs = data[features].values
    inputs = pad_sequences([inputs], maxlen=sequence_length, dtype='float',
                           padding='post', value=-1)[0,:,:]
    xtest_data.append(inputs)

    test_id_data_ = data['ID2'].values
    test_id_data_ = pad_sequences([test_id_data_], maxlen=sequence_length, dtype=object,
                                  padding='post', value='0')[0]
    test_id_data.append(test_id_data_)

xtest_data = np.array(xtest_data)
test_id_data = np.array(test_id_data)

100%|██████████| 476/476 [01:22<00:00,  5.76it/s]


In [18]:
%%time

model_filename = f'network_traffic_model.pth'
batch_size = 16

test_ds = Sequence_Dataset_Inference(xtest_data)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

hidden_size = 64
num_layers = 3
input_size = len(features)
num_classes = len(labels)+1
model = RNN_Model(input_size, num_classes, hidden_size, num_layers).to(device)
model = load_checkpoint(filename=model_filename, model=model)

torch.cuda.empty_cache()
gc.collect()

CPU times: user 253 ms, sys: 995 µs, total: 254 ms
Wall time: 254 ms


9

In [19]:
all_predictions = []
model.eval()
with torch.no_grad():
    pbar = tqdm(enumerate(test_loader), total=len(test_loader), desc='Inference ')
    for step, inputs in pbar:
        inputs = inputs.to(device)
        outputs = model(inputs)
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(gpu_mem=f'{mem:0.2f} GB')

        _, predicted = torch.max(outputs, 2)
        all_predictions.extend(predicted.cpu().numpy().ravel())

Inference : 100%|██████████| 30/30 [00:01<00:00, 25.82it/s, gpu_mem=0.73 GB]


In [20]:
test_id_data_ = test_id_data.ravel()
all_predictions_ = np.array(all_predictions)

sub = pd.DataFrame(all_predictions_)
sub['ID'] = test_id_data_
sub.columns = ['Target','ID']
sub = sub[['ID','Target']]

sub = sub[sub['ID']!='0']
sub.loc[sub['Target']==12,'Target'] = 8
sub_ = pd.merge(ss[['ID']],sub[['ID', 'Target']], how='left', on='ID')

display(sub_)
display(sub_['Target'].value_counts())
sub_.to_csv('sub.csv', index=False)

,ID,Target
0,test0_0,9
1,test0_1,0
2,test0_2,0
3,test0_3,0
4,test0_4,0
...,...,...
2337995,test18_144995,1
2337996,test18_144996,1
2337997,test18_144997,1
2337998,test18_144998,1


11    280232
9     223223
5     221820
6     214771
3     196404
0     194543
7     193779
4     188513
8     181710
2     173643
1     167858
10    101504
Name: Target, dtype: int64